In [1]:
import numpy as np
import pandas as pd

import os
import sys
import itertools

import sklearn as sk
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import xgboost as xgb
import scipy.stats as stats
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from scipy.optimize import minimize

import matplotlib.pyplot as plt

from datetime import timedelta
import holidays

import enefit

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
env = enefit.make_env()

# ETL

## Load & Convert Data

In [4]:
df_client = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/client.csv")
df_electricity = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv")
df_gas = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv")
df_weather = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv")
df_train = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")
df_stations_county_mapping = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv")

In [5]:
df_train['datetime'] = pd.to_datetime(df_train['datetime'])

df_medianes = df_train.loc[((df_train['datetime']>='2022-01-01') & (df_train['datetime']<='2022-04-30')) | 
                           ((df_train['datetime']>='2023-01-01') & (df_train['datetime']<='2023-04-30'))].copy()

spring_consumption = df_medianes[['target']].loc[df_medianes.is_consumption==1].median()
med_spring_consumption = spring_consumption[0]
spring_production = df_medianes[['target']].loc[df_medianes.is_consumption==0].median()
med_spring_production = spring_production[0]

print("Median Spring Consumption : {:.2f}, Production : {:.2f}".format(med_spring_consumption, med_spring_production))

Median Spring Consumption : 137.83, Production : 0.17


mapping, **reference dataframe**

In [6]:
# Fill empty values of county with 12. County 12 is set as 'unknown', so we will average all 'unknown' locations into 1
df_stations_county_mapping['county'].fillna(12, inplace=True)

# primary key
df_stations_county_mapping['cle_lat-long'] = round(df_stations_county_mapping.latitude, 1).astype(str) + "-" + df_stations_county_mapping.longitude.astype(str)

df_stations_county_mapping = df_stations_county_mapping[['cle_lat-long','county']]

Weather forecast

In [7]:
# Keeps only forecast for the next day
df_weather =  df_weather.loc[(df_weather.hours_ahead>=22) & (df_weather.hours_ahead<=45)]
# primary key
df_weather['cle_lat-long'] = df_weather.latitude.astype(str) + "-" + df_weather.longitude.astype(str)

In [8]:
# Join to add county to each lat-long key
df_forecast_w_per_county = pd.merge(df_weather, df_stations_county_mapping, on='cle_lat-long')

In [9]:
# Agg measures per county per hour
# as_index= False to preserve hourly measures
df_forecast_w_per_county = df_forecast_w_per_county.groupby(['forecast_datetime', 'county'], as_index=False)\
.agg({'temperature': ['min', 'mean', 'max'],\
      'dewpoint': 'mean',\
      '10_metre_u_wind_component': 'mean',\
      'direct_solar_radiation': ['mean', 'max'],\
      'surface_solar_radiation_downwards' : ['mean', 'max']
     })

# Flatten multi-level columns
df_forecast_w_per_county.columns = ["_".join(a) for a in df_forecast_w_per_county.columns.to_flat_index()]
df_forecast_w_per_county.rename(columns={'forecast_datetime_':'datetime','county_':'county'}, inplace=True)

In [10]:
df_forecast_w_per_county['county'] = df_forecast_w_per_county.county.astype(int)
df_forecast_w_per_county['datetime'] = pd.to_datetime(df_forecast_w_per_county['datetime'], format='%Y-%m-%d %H:%M:%S')
df_forecast_w_per_county['datetime'] = df_forecast_w_per_county['datetime'].dt.tz_localize('UTC')

Keep forecast_datetime as object or reduced to day precision after merge or groupby

Client

In [11]:
df_client["date"] = pd.to_datetime(df_client.date)
# Date where the measure is received
# Converts to str to join as key later
#df_client["datetime_day"] = (df_client["date"] + timedelta(days=2)).dt.strftime('%Y-%m-%d')
df_client["datetime_day"] = (df_client["date"] + timedelta(days=2)).dt.tz_localize('UTC')

df_client = df_client[['datetime_day','county','is_business','product_type','eic_count','installed_capacity']]

gas et elec nothing to change. Forecast_date already set, as object.<br>
Beware : electricity/h, gas/d

In [12]:
df_gas.rename(columns={'forecast_date':'datetime_day'}, inplace=True)

df_gas["datetime_day"] = pd.to_datetime(df_gas.datetime_day)
#df_gas["datetime_day"] = (df_gas["date"] + timedelta(days=1)).dt.strftime('%Y-%m-%d')
df_gas["datetime_day"] = (df_gas["datetime_day"] + timedelta(days=1)).dt.tz_localize('UTC')

df_gas = df_gas[['datetime_day','lowest_price_per_mwh','highest_price_per_mwh']]

In [13]:
df_electricity['datetime'] = pd.to_datetime(df_electricity['forecast_date'], format='%Y-%m-%d %H:%M:%S')
df_electricity['datetime'] = (df_electricity['datetime'] + timedelta(days=1)).dt.tz_localize('UTC')
df_electricity = df_electricity[['datetime','euros_per_mwh']]

## Fourier transform and out-of-sample

We want to isolate only pid where measure are complete for training.<br>
Extrapolate on missing data would have been possible, but data is sufficient for proper training.<br>
Some pid (68 i.e.) have way too much missing values, we want to keep them away.

In [14]:
df_conso = df_train[df_train.is_consumption==1].copy()

df_full_pid = df_conso.groupby('prediction_unit_id').size().reset_index(name='count')

pid_is_complete = df_full_pid['count'].max()
df_full_pid = df_full_pid[df_full_pid['count'] == pid_is_complete]

print("{} prediction_unit_id are complete out of 69".format(df_full_pid.shape[0]))

del df_conso

56 prediction_unit_id are complete out of 69


In [15]:
# Some target are nulls.
# They are observed during daytime change, summer and winter
df_train[['datetime']].loc[df_train.target.isnull()].value_counts()

datetime           
2022-10-30 03:00:00    136
2022-03-27 03:00:00    134
2023-03-26 03:00:00    132
2021-10-31 03:00:00    126
Name: count, dtype: int64

In [16]:
# We can drop them, no need to interpolate
df_train.dropna(subset=['target'], inplace=True)

In [17]:
df_full_pid_conso = df_train.loc[df_train.is_consumption==1].merge(\
                        df_full_pid[['prediction_unit_id']], on='prediction_unit_id', how='inner')

df_full_pid_prod = df_train.loc[df_train.is_consumption==0].merge(\
                        df_full_pid[['prediction_unit_id']], on='prediction_unit_id', how='inner')

#### Functions definition

In [18]:
def fourier_features(index, freq, order):
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order + 1):
        features.update({
            f"sin_{freq}_{i}": np.sin(i * k),
            f"cos_{freq}_{i}": np.cos(i * k),
        })
    return pd.DataFrame(features, index=index)

In [19]:
def add_season(df, consumption, group):
    ''' Adds the seasonal Fourier in and out of sample, up to the end of the competition, per hour'''
    
    df_fourier = df.loc[ df.prediction_unit_id == group ]
    # str -> datetime, then as index
    df_fourier['datetime'] = pd.to_datetime(df_fourier['datetime'], format='%Y-%m-%d %H:%M:%S')
    df_fourier = df_fourier.set_index('datetime').sort_index(ascending=True)
    # remove timezone aware from index
    df_fourier.index = df_fourier.index.tz_localize(None)
    
    # Orders are not the same for production and consumption.
    # See periodogram in the dedicated notebook
    nb_f = 4 if consumption==True else 3
    
    fourier = CalendarFourier(freq="A", order=nb_f)

    dp = DeterministicProcess(
        # Need to pass index as period, otherwise won't work
        index=df_fourier.index.to_period("H"), #slice the index with in sample only
        constant=True,               # dummy feature for bias (y-intercept)
        order=1,                     # trend (order 1 means linear)
        seasonal=True,               # weekly seasonality (indicators)
        additional_terms=[fourier],  # annual seasonality (fourier)
        drop=True                    # drop terms to avoid collinearity
    )

    # in-sample
    X = dp.in_sample()
    y = df_fourier['target']

    model = LinearRegression(fit_intercept=False)
    _ = model.fit(X, y)

    y_pred = pd.Series(model.predict(X), index=y.index)

    # There are 5880 more rows between 31-05-2022 and 02-06-2024
    # This should be the largest 'real data' test possible
    X_fore = dp.out_of_sample(steps=8820)
    y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

    # Concatenate in and out of sample
    y_fore.index = y_fore.index.to_timestamp()
    y_season = pd.concat([y_pred, y_fore])

    df_season = pd.DataFrame(y_season, columns=['seasonality'])
    
    df_fourier = df_fourier.join(df_season, how='right')
    # Fill pid for out of sample values
    df_fourier['prediction_unit_id'] = group
    df_fourier['is_consumption'] = 1 if consumption==True else 0


    return df_fourier

In [20]:
def add_median_columns(df_in):
    
    # Consumption and production can't be below 0. Floor Fourier functions to 0.
    #df_in.loc[df_in.seasonality < 0, 'seasonality'] = 0

    # Seasonality per county
    df_gp = df_in.groupby(['datetime','county'])['seasonality'].median().reset_index()
    df_gp.rename(columns={'seasonality':'med_st_county'}, inplace=True)
    df_in = pd.merge(df_in, df_gp, on=['datetime', 'county'])

    # Seasonality per business
    df_gp = df_in.groupby(['datetime','is_business'])['seasonality'].median().reset_index()
    df_gp.rename(columns={'seasonality':'med_st_business'}, inplace=True)
    df_in = pd.merge(df_in, df_gp, on=['datetime', 'is_business'])

    # Seasonality per product
    df_gp = df_in.groupby(['datetime','product_type'])['seasonality'].median().reset_index()
    df_gp.rename(columns={'seasonality':'med_st_product_type'}, inplace=True)
    df_in = pd.merge(df_in, df_gp, on=['datetime', 'product_type'])

    return df_in

In [21]:
def optim_coeffs(df_in):
    
    df_optim = df_in[df_in['target'].notna()]

    # Define the coefficients
    x = np.random.rand()
    y = np.random.rand()
    z = np.random.rand()
    
    def f(coefficients):
        ''' Define the function to minimize'''
        x, y, z = coefficients
        return np.sum((x * df_optim['med_st_county'] + y * df_optim['med_st_business'] + z * df_optim['med_st_product_type'] - df_optim['target']) ** 2)
    
    # Minimize the function
    res = minimize(f, [x, y, z])

    x_opt = res.x[0]
    y_opt = res.x[1]
    z_opt = res.x[2]
    coeffs = [x_opt, y_opt, z_opt]

    df_in['st_target'] = x_opt*df_in.med_st_county + y_opt*df_in.med_st_business + z_opt*df_in.med_st_product_type
    df_in['residuals'] = df_in.target - df_in.st_target
    
    return df_in, coeffs

In [22]:
def extract_date_components(df_in):
    '''Feature engineering for dates.
    Add datetime day, holiday, and datetime components'''
    
    df_in['datetime_day'] = df_in['datetime'].dt.date
    df_in['datetime_day'] = pd.to_datetime(df_in['datetime_day'], format='%Y-%m-%d')
    
    # Add Estonian holiday column, based on the previous one
    est_holidays = holidays.Estonia(years=[2021, 2022, 2023, 2024])
    df_in['holiday'] = df_in['datetime_day'].apply(lambda x: 1 if x in est_holidays else 0)
    
    # Create new columns from date column
    df_in['week_of_year'] = df_in['datetime'].dt.isocalendar().week.astype(int)
    df_in['day_of_week'] = df_in['datetime'].dt.dayofweek.astype(int)
    df_in['day_of_year'] = df_in['datetime'].dt.dayofyear.astype(int)
    df_in['hour_of_day'] = df_in['datetime'].dt.hour.astype(int)
    df_in['month'] = df_in['datetime'].dt.month.astype(int)
    
    return df_in

In [23]:
def run_fourier_and_optim(df_in, consumption, pid_list): 
    
    df_list = []

    for p in pids:
        df_list.append( add_season(df_in, consumption=consumption, group=p) )
    
    df_fourier = pd.concat(df_list)

    # Datetime back as column, no timezone
    df_fourier.reset_index(inplace=True)
    df_fourier.rename(columns={'index':'datetime'},inplace=True)
    df_fourier['datetime'] = df_fourier['datetime'].dt.tz_localize('UTC')
    
    # Feature Engineering for datetime column
    df_fourier = extract_date_components(df_fourier)
    df_fourier['datetime_day'] = df_fourier['datetime_day'].dt.tz_localize('UTC')
    
    # Add 'county','is_business','product_type' to out of sample. Join with distinct pid combinations
    df_fourier = pd.merge(df_fourier.drop(columns=['county','is_business','product_type']), df_distinct, on=['prediction_unit_id'], how='left')

    # Median columns calculation
    df_fourier = add_median_columns(df_fourier)
    
    # Coeff optimisation
    df_fourier, coeffs = optim_coeffs(df_fourier)

    return df_fourier.drop(columns=['data_block_id']), coeffs

In [24]:
def unite_dfs(df_in, client, gas, elec, weather_forecast):
    ''' Merge all DataFrames into one'''
    
    df_unite = pd.merge(df_in, client, on=['datetime_day', 'county', 'is_business', 'product_type'], how='left')
    df_unite = pd.merge(df_unite, gas, on=['datetime_day'], how='left')
    df_unite = pd.merge(df_unite, elec, on=['datetime'], how='left')
    df_unite = pd.merge(df_unite, weather_forecast, on=['datetime','county'], how='left')
    
    return df_unite

In [25]:
def generate_FE_columns(df_in):

    df_in['ssrd_mean*temp_mean'] = df_in.surface_solar_radiation_downwards_mean * df_in.temperature_mean
    df_in['ssrd_mean*temp_max'] = df_in.surface_solar_radiation_downwards_mean * df_in.temperature_max
    df_in['ssrd_mean*hod'] = df_in.surface_solar_radiation_downwards_mean * df_in.hour_of_day
    df_in['ssrd_mean*eic'] = df_in.surface_solar_radiation_downwards_mean * df_in.eic_count
    df_in['ssrd_max*eic'] = df_in.surface_solar_radiation_downwards_max * df_in.eic_count
    df_in['ssrd_mean*capacity'] = df_in.surface_solar_radiation_downwards_mean * df_in.installed_capacity
    df_in['ssrd_max*capacity'] = df_in.surface_solar_radiation_downwards_max * df_in.installed_capacity
    df_in['ssrd_mean*dsr_mean'] = df_in.surface_solar_radiation_downwards_mean * df_in.direct_solar_radiation_mean
    df_in['ssrd_max*dsr_mean'] = df_in.surface_solar_radiation_downwards_max * df_in.direct_solar_radiation_mean
    
    df_in['temp_max*hod'] = df_in.temperature_max * df_in.hour_of_day
    df_in['temp_max*dsr_max'] = df_in.temperature_max * df_in.direct_solar_radiation_max
    df_in['temp_min*capacity'] = df_in.temperature_min * df_in.installed_capacity
    
    df_in['dsr_mean*dsr_max'] = df_in.direct_solar_radiation_mean * df_in.direct_solar_radiation_max
    
    df_in['capacity*business'] = df_in.installed_capacity * df_in.is_business
    df_in['holiday*eic'] = df_in.holiday * df_in.eic_count
    
    df_in['sqrt_capacity'] = df_in.installed_capacity ** 0.5
    
    return df_in

In [26]:
def interpolate_per_pids(df, group):
    '''Interpolate null Target values.
    Per given group'''
    
    # Handle on selected group, in-sample data, for training
    df_extract = df.loc[df['prediction_unit_id'] == group].copy()

    # Datetime has to be passed to index to interpolate
    df_extract.set_index('datetime', inplace=True)
    df_extract.sort_index(ascending=True, inplace=True)

    # The pad method fills the missing values with the last known value in the column.
    df_extract.interpolate(method='pad', inplace=True)

    # Bring datetime back to column. Easier for future pd.contact for each group
    return df_extract.reset_index()


#### Run Fourier transform

In [27]:
d_dfs_in = {
    "df_in":[df_full_pid_conso, df_full_pid_prod],
    "consumption":[True, False]
}

# List of all unique pids
pids = set(df_full_pid_conso.prediction_unit_id)

df_distinct = df_train[['prediction_unit_id','county','is_business','product_type']].drop_duplicates()

In [28]:
# Add Fourier Season + Trend /county /isBusiness /Product, on prod & conso datasets separately
df_ref_conso, coeffs_conso = run_fourier_and_optim(d_dfs_in['df_in'][0], d_dfs_in['consumption'][0], pids)
df_ref_prod, coeffs_prod = run_fourier_and_optim(d_dfs_in['df_in'][1], d_dfs_in['consumption'][1], pids)

In [29]:
print(coeffs_conso)
print(coeffs_prod)

[1.60664488843706, 1.0218816343893593, -0.07560426762620513]
[1.834457779476791, -2.829650187844042, 1.8828243653414085]


In [30]:
# Merge main DataFrames with additional Measures Datasets

df_raw_conso = unite_dfs(df_ref_conso.loc[(df_ref_conso.datetime_day>='2021-09-03') & (df_ref_conso.datetime_day<='2023-05-31')],
                           df_client, df_gas, df_electricity, df_forecast_w_per_county)
     
df_raw_prod = unite_dfs(df_ref_prod.loc[(df_ref_prod.datetime_day>='2021-09-03') & (df_ref_prod.datetime_day<='2023-05-31')],
                          df_client, df_gas, df_electricity, df_forecast_w_per_county)

In [31]:
# extract in-sample data, and interpolate values per PIDs

col_list = df_raw_conso.columns.to_list()
df_train_conso = pd.DataFrame(columns=col_list)
df_train_prod = pd.DataFrame(columns=col_list)

for g in list(pids):
        df_train_conso = pd.concat([df_train_conso, interpolate_per_pids(df_raw_conso, group=g)])
        df_train_prod = pd.concat([df_train_prod, interpolate_per_pids(df_raw_prod, group=g)])

del df_raw_conso, df_raw_prod

In [32]:
# Convert columns incorrectly converted as object back to int
obj_cols = df_train_conso.drop('datetime_day', axis=1).select_dtypes(include=[object]).columns

df_train_conso[obj_cols] = df_train_conso[obj_cols].astype(int)
df_train_prod[obj_cols] = df_train_prod[obj_cols].astype(int)

In [33]:
# Add Feature Engineering columns
df_train_conso = generate_FE_columns(df_train_conso)
df_train_prod = generate_FE_columns(df_train_prod)

In [34]:
df_train_conso.isnull().sum()

datetime                                  0
target                                    0
is_consumption                            0
row_id                                    0
prediction_unit_id                        0
seasonality                               0
datetime_day                              0
holiday                                   0
week_of_year                              0
day_of_week                               0
day_of_year                               0
hour_of_day                               0
month                                     0
county                                    0
is_business                               0
product_type                              0
med_st_county                             0
med_st_business                           0
med_st_product_type                       0
st_target                                 0
residuals                                 0
eic_count                                 0
installed_capacity              

In [35]:
df_train_prod.isnull().sum()

datetime                                  0
target                                    0
is_consumption                            0
row_id                                    0
prediction_unit_id                        0
seasonality                               0
datetime_day                              0
holiday                                   0
week_of_year                              0
day_of_week                               0
day_of_year                               0
hour_of_day                               0
month                                     0
county                                    0
is_business                               0
product_type                              0
med_st_county                             0
med_st_business                           0
med_st_product_type                       0
st_target                                 0
residuals                                 0
eic_count                                 0
installed_capacity              

# Train & Fit Model

Stratify per month before train, in order to have the closest 'target' distribution on each set<br> *(train, test, validate. For CONSO & PROD)*.<br>
We also focus the training on months to forecast

In [36]:
#df_train_conso['month'] = df_train_conso['datetime'].dt.month.astype(int)
#df_train_prod['month'] = df_train_prod['datetime'].dt.month.astype(int)

df_spring_conso = df_train_conso.loc[df_train_conso['month'].isin(list(range(1,5)) + list(range(9,13)))].copy()
df_spring_prod = df_train_prod.loc[df_train_prod['month'].isin(list(range(1,5)) + list(range(9,13)))].copy()

In [37]:
# 70/20/10 en conservant la même ratio de mois pour chaque.
# target a trop de valeurs uniques, datetime_day serait trop fin, woy bizarre sur semaine 35

X_train_conso, test_validate_conso = train_test_split(df_spring_conso, test_size=0.3, random_state=1334, stratify=df_spring_conso['month'])
X_test_conso, X_validate_conso = train_test_split(test_validate_conso, test_size=0.33, random_state=1334, stratify=test_validate_conso['month'])

X_train_prod, test_validate_prod = train_test_split(df_spring_prod, test_size=0.3, random_state=1334, stratify=df_spring_prod['month'])
X_test_prod, X_validate_prod = train_test_split(test_validate_prod, test_size=0.33, random_state=1334, stratify=test_validate_prod['month'])

In [38]:
# extract y_train, y_test conso

y_train_conso = X_train_conso['residuals']
y_test_conso = X_test_conso['residuals']
y_validate_conso = X_validate_conso['residuals']

X_train_conso = X_train_conso.drop(columns=['datetime','row_id','is_consumption','datetime_day','target','prediction_unit_id','seasonality','med_st_county','med_st_business','med_st_product_type','st_target','residuals'])
X_test_conso = X_test_conso.drop(columns=['datetime','row_id','is_consumption','datetime_day','target','prediction_unit_id','seasonality','med_st_county','med_st_business','med_st_product_type','st_target','residuals'])
X_validate_conso = X_validate_conso.drop(columns=['datetime','row_id','is_consumption','datetime_day','target','prediction_unit_id','seasonality','med_st_county','med_st_business','med_st_product_type','st_target','residuals'])

# & prod
y_train_prod = X_train_prod['residuals']
y_test_prod = X_test_prod['residuals']
y_validate_prod = X_validate_prod['residuals']

X_train_prod = X_train_prod.drop(columns=['datetime','row_id','is_consumption','datetime_day','target','prediction_unit_id','seasonality','med_st_county','med_st_business','med_st_product_type','st_target','residuals'])
X_test_prod = X_test_prod.drop(columns=['datetime','row_id','is_consumption','datetime_day','target','prediction_unit_id','seasonality','med_st_county','med_st_business','med_st_product_type','st_target','residuals'])
X_validate_prod = X_validate_prod.drop(columns=['datetime','row_id','is_consumption','datetime_day','target','prediction_unit_id','seasonality','med_st_county','med_st_business','med_st_product_type','st_target','residuals'])

In [39]:
# Compare the distributions of the target columns
statistic, pvalue = stats.ks_2samp(y_train_conso, y_test_conso)
print(f"p-value conso train/test is {pvalue:.4f}")
statistic, pvalue = stats.ks_2samp(y_train_conso, y_validate_conso)
print(f"p-value conso train/validate is {pvalue:.4f}")
statistic, pvalue = stats.ks_2samp(y_test_conso, y_validate_conso)
print(f"p-value conso test/validate is {pvalue:.4f}")

print("=============================================")

statistic, pvalue = stats.ks_2samp(y_train_prod, y_test_prod)
print(f"p-value prod train/test is {pvalue:.4f}")
statistic, pvalue = stats.ks_2samp(y_train_prod, y_validate_prod)
print(f"p-value prod train/validate is {pvalue:.4f}")
statistic, pvalue = stats.ks_2samp(y_test_prod, y_validate_prod)
print(f"p-value prod test/validate is {pvalue:.4f}")


p-value conso train/test is 0.8441
p-value conso train/validate is 0.8644
p-value conso test/validate is 0.6508
p-value prod train/test is 0.3100
p-value prod train/validate is 0.1372
p-value prod test/validate is 0.5223


#### MAE pour conso

In [40]:
clf_conso = LGBMRegressor(
                    boosting_type='gbdt',
                    num_leaves=500,
                    max_depth=20,
                    learning_rate=0.407758307054,
                    n_estimators=301,
                    subsample_for_bin=200000,
                    min_split_gain=1.0,
                    min_child_weight=0.001,
                    min_child_samples=1,
                    subsample=0.9,
                    subsample_freq=1,
                    colsample_bytree=1.0,
                    reg_alpha=0.637303362873,
                    reg_lambda=1.0,
                    random_state=1337
                  )

clf = xgb.XGBRegressor(
                    max_depth=6,
                    learning_rate=0.2249351,
                    gamma=0.14525607,
                    min_child_weight=2.4945118,
                    max_delta_step=0.0,
                    subsample=0.6806308,
                    colsample_bytree=0.5475739,
                    colsample_bylevel=0.9011847,
                    reg_alpha=0.78955835,
                    reg_lambda=0.7961774,
                    n_estimators=9996,
                    silent=0,
                    scale_pos_weight=1.0,
                    base_score=0.5,
                    seed=1664
                  )

In [41]:
%time clf_conso.fit(X_train_conso, y_train_conso)

CPU times: user 1min 45s, sys: 20.3 s, total: 2min 5s
Wall time: 54.3 s


LGBMRegressor(learning_rate=0.407758307054, max_depth=20, min_child_samples=1,
              min_split_gain=1.0, n_estimators=301, num_leaves=500,
              random_state=1337, reg_alpha=0.637303362873, reg_lambda=1.0,
              subsample=0.9, subsample_freq=1)

In [42]:
%time _predictions_conso = clf_conso.predict(X_test_conso)
predictions_conso = pd.Series(data=_predictions_conso, index=X_test_conso.index, name='predicted')

CPU times: user 5.78 s, sys: 12.5 ms, total: 5.79 s
Wall time: 1.5 s


In [43]:
%time _validate_conso = clf_conso.predict(X_validate_conso)
validate_conso = pd.Series(data=_validate_conso, index=X_validate_conso.index, name='validation')

CPU times: user 2.84 s, sys: 289 µs, total: 2.84 s
Wall time: 739 ms


In [44]:
mae_conso = mean_absolute_error(y_test_conso, predictions_conso)
print("MAE Test : ",mae_conso)
mae_validate = mean_absolute_error(y_validate_conso, validate_conso)
print("MAE validate : ",mae_validate)

MAE Test :  35.310207334952764
MAE validate :  35.581047369431275


#### MAE pour prod

In [45]:
clf_prod = LGBMRegressor(
                    boosting_type='gbdt',
                    num_leaves=500,
                    max_depth=20,
                    learning_rate=0.155242982923,
                    n_estimators=301,
                    subsample_for_bin=200000,
                    min_split_gain=0.0,
                    min_child_weight=0.001,
                    min_child_samples=1,
                    subsample=0.9,
                    subsample_freq=1,
                    colsample_bytree=1.0,
                    reg_alpha=0.0,
                    reg_lambda=1.0,
                    random_state=1337
)

In [46]:
%time clf_prod.fit(X_train_prod, y_train_prod)

CPU times: user 1min 49s, sys: 20.4 s, total: 2min 9s
Wall time: 55.2 s


LGBMRegressor(learning_rate=0.155242982923, max_depth=20, min_child_samples=1,
              n_estimators=301, num_leaves=500, random_state=1337,
              reg_lambda=1.0, subsample=0.9, subsample_freq=1)

In [47]:
%time _predictions_prod = clf_prod.predict(X_test_prod)
predictions_prod = pd.Series(data=_predictions_prod, index=X_test_prod.index, name='predicted')

CPU times: user 6.58 s, sys: 0 ns, total: 6.58 s
Wall time: 1.7 s


In [48]:
%time _validate_prod = clf_prod.predict(X_validate_prod)
validate_prod = pd.Series(data=_validate_prod, index=X_validate_prod.index, name='validation')

CPU times: user 3.25 s, sys: 0 ns, total: 3.25 s
Wall time: 836 ms


In [49]:
mae_prod = mean_absolute_error(y_test_prod, predictions_prod)
print("MAE test prod : ",mae_prod)
mae_validate_prod = mean_absolute_error(y_validate_prod, validate_prod)
print("MAE validate prod: ",mae_validate_prod)

MAE test prod :  17.787692884941066
MAE validate prod:  17.92961004607065


# Test set

In [50]:
class run_ETL:
    
    def __init__(self, client, elec, gas, weather, main):
        self.df_client = client
        self.df_electricity = elec
        self.df_gas = gas
        self.df_weather = weather
        self.df_test = main
        
    
    def transform_measures(self):
        
        # ======================== WEATHER FORECAST ============================================
        # Keeps only forecast for the next day
        self.df_weather =  self.df_weather.loc[(self.df_weather.hours_ahead>=22) & (self.df_weather.hours_ahead<=45)]
        # primary key
        self.df_weather['cle_lat-long'] = self.df_weather.latitude.astype(str) + "-" + self.df_weather.longitude.astype(str)
        # Join to add county to each lat-long key
        self.df_forecast_w_per_county = pd.merge(self.df_weather, df_stations_county_mapping, on='cle_lat-long')

        # Agg measures per county per hour
        # as_index= False to preserve hourly measures
        self.df_forecast_w_per_county = self.df_forecast_w_per_county.groupby(['forecast_datetime', 'county'], as_index=False)\
        .agg({'temperature': ['min', 'mean', 'max'],\
              'dewpoint': 'mean',\
              '10_metre_u_wind_component': 'mean',\
              'direct_solar_radiation': ['mean', 'max'],\
              'surface_solar_radiation_downwards' : ['mean', 'max']
             })

        # Flatten multi-level columns
        self.df_forecast_w_per_county.columns = ["_".join(a) for a in self.df_forecast_w_per_county.columns.to_flat_index()]
        self.df_forecast_w_per_county.rename(columns={'forecast_datetime_':'datetime','county_':'county'}, inplace=True)
        self.df_forecast_w_per_county['county'] = self.df_forecast_w_per_county.county.astype(int)
        self.df_forecast_w_per_county['datetime'] = pd.to_datetime(self.df_forecast_w_per_county['datetime'], format='%Y-%m-%d %H:%M:%S')
        self.df_forecast_w_per_county['datetime'] = self.df_forecast_w_per_county['datetime'].dt.tz_localize('UTC')
        
        # ======================== CLIENT ============================================
        self.df_client["date"] = pd.to_datetime(self.df_client.date)
        # Date where the measure is received, adjusted to prediction datetime
        self.df_client["datetime_day"] = (self.df_client["date"] + timedelta(days=2)).dt.tz_localize('UTC')
        self.df_client = self.df_client[['datetime_day','county','is_business','product_type','eic_count','installed_capacity']]
        
        # ======================== GAS  & ELEC ============================================
        self.df_gas.rename(columns={'forecast_date':'datetime_day'}, inplace=True)
        self.df_gas['datetime_day'] = pd.to_datetime(self.df_gas['datetime_day'])
        self.df_gas['datetime_day'] = (self.df_gas['datetime_day'] + timedelta(days=1)).dt.tz_localize('UTC')
        self.df_gas = self.df_gas[['datetime_day','lowest_price_per_mwh','highest_price_per_mwh']]
        
        self.df_electricity['datetime'] = pd.to_datetime(self.df_electricity['forecast_date'], format='%Y-%m-%d %H:%M:%S')
        self.df_electricity['datetime'] = (self.df_electricity['datetime'] + timedelta(days=1)).dt.tz_localize('UTC')
        self.df_electricity = self.df_electricity[['datetime','euros_per_mwh']]
        
    
    def split_and_add_fourier_forecast(self):
        
        #self.df_test.rename(columns={'prediction_datetime':'datetime'}, inplace=True)
        self.df_test['datetime'] = pd.to_datetime(self.df_test['prediction_datetime'], format='%Y-%m-%d %H:%M:%S')
        self.df_test['datetime'] = self.df_test['datetime'].dt.tz_localize('UTC')
        
        self.df_conso_test = self.df_test[self.df_test.is_consumption==1].copy()
        # Join aggregates columns
        self.df_conso_test = pd.merge(self.df_conso_test, df_ref_conso[['datetime','county','med_st_county']].drop_duplicates(), on=['datetime','county'], how='left')
        self.df_conso_test = pd.merge(self.df_conso_test, df_ref_conso[['datetime','is_business','med_st_business']].drop_duplicates(), on=['datetime','is_business'], how='left')
        self.df_conso_test = pd.merge(self.df_conso_test, df_ref_conso[['datetime','product_type','med_st_product_type']].drop_duplicates(), on=['datetime','product_type'], how='left')
        self.df_conso_test = pd.merge(self.df_conso_test, df_ref_conso[['datetime','datetime_day','holiday','week_of_year','day_of_week','day_of_year','hour_of_day','month']].drop_duplicates(), on=['datetime'], how='left')
        # Compute seasonnalty + trend based on coefficients 
        self.df_conso_test['st_target'] = coeffs_conso[0]*self.df_conso_test.med_st_county + coeffs_conso[1]*self.df_conso_test.med_st_business + coeffs_conso[2]*self.df_conso_test.med_st_product_type

        self.df_prod_test = self.df_test[self.df_test.is_consumption==0].copy()
        self.df_prod_test = pd.merge(self.df_prod_test, df_ref_prod[['datetime','county','med_st_county']].drop_duplicates(), on=['datetime','county'], how='left')
        self.df_prod_test = pd.merge(self.df_prod_test, df_ref_prod[['datetime','is_business','med_st_business']].drop_duplicates(), on=['datetime','is_business'], how='left')
        self.df_prod_test = pd.merge(self.df_prod_test, df_ref_prod[['datetime','product_type','med_st_product_type']].drop_duplicates(), on=['datetime','product_type'], how='left')
        self.df_prod_test = pd.merge(self.df_prod_test, df_ref_prod[['datetime','datetime_day','holiday','week_of_year','day_of_week','day_of_year','hour_of_day','month']].drop_duplicates(), on=['datetime'], how='left')
        # Compute seasonnalty + trend based on coefficients 
        self.df_prod_test['st_target'] = coeffs_prod[0]*self.df_prod_test.med_st_county + coeffs_prod[1]*self.df_prod_test.med_st_business + coeffs_prod[2]*self.df_prod_test.med_st_product_type

    
    def unite_FE_and_interpolate(self):
        
        # List of all unique pids
        pids = set(self.df_conso_test.prediction_unit_id)
        
        # Join all dataframes together
        self.df_conso_test = unite_dfs(self.df_conso_test, self.df_client, self.df_gas, self.df_electricity, self.df_forecast_w_per_county)
        self.df_prod_test = unite_dfs(self.df_prod_test, self.df_client, self.df_gas, self.df_electricity, self.df_forecast_w_per_county)
        
        # Interpolate nulls values (prices elec & gas are concerned on last day)
        col_list = self.df_prod_test.columns.to_list()
        self.df_conso_test_toPredict = pd.DataFrame(columns=col_list)
        self.df_prod_test_toPredict = pd.DataFrame(columns=col_list)

        for g in list(pids):
            self.df_conso_test_toPredict = pd.concat([self.df_conso_test_toPredict, interpolate_per_pids(self.df_conso_test, group=g)])
            self.df_prod_test_toPredict = pd.concat([self.df_prod_test_toPredict, interpolate_per_pids(self.df_prod_test, group=g)])
        
        # Convert columns incorrectly converted as object back to int
        obj_cols = self.df_conso_test_toPredict.drop(columns=['prediction_datetime','datetime_day']).select_dtypes(include=[object]).columns
        self.df_conso_test_toPredict[obj_cols] = self.df_conso_test_toPredict[obj_cols].astype(int)
        self.df_prod_test_toPredict[obj_cols] = self.df_prod_test_toPredict[obj_cols].astype(int)
        
        # Add Feature Engineering columns
        self.df_conso_test_toPredict = generate_FE_columns(self.df_conso_test_toPredict)
        self.df_prod_test_toPredict = generate_FE_columns(self.df_prod_test_toPredict)
    
    
    def predict_residuals_and_target(self):
        
        # Keep only necessary columns, the model has been trained on
        cols_to_keep = X_test_conso.columns.to_list()
        # index is a loop since interpolate -> need to reset it before predict and join results
        self.df_conso_test_toPredict.reset_index(drop=True, inplace=True)
        self.df_prod_test_toPredict.reset_index(drop=True, inplace=True)
        
        _predictions_conso = clf_conso.predict(self.df_conso_test_toPredict[cols_to_keep])
        predictions_conso = pd.Series(data=_predictions_conso, index=self.df_conso_test_toPredict.index, name='predicted_residuals')
        self.df_conso_test_toPredict = self.df_conso_test_toPredict.join(predictions_conso, how='left')
        self.df_conso_test_toPredict['predicted_target'] = self.df_conso_test_toPredict['st_target'] + self.df_conso_test_toPredict['predicted_residuals']
        # Replace negative values by 0
        self.df_conso_test_toPredict['predicted_target'] = self.df_conso_test_toPredict['predicted_target'].clip(lower=med_spring_consumption)
        
        _predictions_prod = clf_prod.predict(self.df_prod_test_toPredict[cols_to_keep])
        predictions_prod = pd.Series(data=_predictions_prod, index=self.df_prod_test_toPredict.index, name='predicted_residuals')
        self.df_prod_test_toPredict = self.df_prod_test_toPredict.join(predictions_prod, how='left')
        self.df_prod_test_toPredict['predicted_target'] = self.df_prod_test_toPredict['st_target'] + self.df_prod_test_toPredict['predicted_residuals']
        self.df_prod_test_toPredict['predicted_target'] = self.df_prod_test_toPredict['predicted_target'].clip(lower=med_spring_production)
    
    
    def merge_and_submit(self):
        
        self.df_concat = pd.concat([etl.df_conso_test_toPredict, etl.df_prod_test_toPredict]).sort_values(by='row_id')
        self.df_test = pd.merge(self.df_test, self.df_concat[['row_id','predicted_target']], on=['row_id'], how='left')

# Submit

In [51]:
iter_test = env.iter_test()

In [52]:
def is_prediciton_needed(df):
    ''' Update of competition submission rules.
    Scores only if test column currently_scored is set to True.'''
    return not all(df['currently_scored'] == False)

In [53]:
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    
    etl = run_ETL(client = client,
             elec = electricity_prices,
             gas = gas_prices,
             weather = forecast_weather,
             main = test)
    
    etl.transform_measures()
    etl.split_and_add_fourier_forecast()
    etl.unite_FE_and_interpolate()
    etl.predict_residuals_and_target()
    etl.merge_and_submit()
    
    #df_submit = etl.df_test[['row_id','predicted_target']].sort_values(by=['row_id'], ascending=True).copy()
    #df_submit['predicted_target'] = round(df_submit['predicted_target'], 4)
    #target = df_submit['predicted_target'].to_numpy()
    
    if not is_prediciton_needed(test):
        print("all ['currently_scored'] values are False -> dummy Prediction 0")
        sample_prediction['target'] = 0
        env.predict(sample_prediction)
        continue
    
    #sample_prediction['target'] = target
    sample_prediction = pd.merge(sample_prediction.drop(columns='target'), etl.df_test[['row_id','predicted_target']].rename(columns={'predicted_target':'target'}), on=['row_id'], how='left')
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
all ['currently_scored'] values are False -> dummy Prediction 0
all ['currently_scored'] values are False -> dummy Prediction 0
all ['currently_scored'] values are False -> dummy Prediction 0
all ['currently_scored'] values are False -> dummy Prediction 0
